# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4bacbf9fa0>
├── 'a' --> tensor([[ 0.4342, -0.1405, -2.2848],
│                   [-0.5769,  0.6934, -0.6881]])
└── 'x' --> <FastTreeValue 0x7f4bacc0d160>
    └── 'c' --> tensor([[ 0.4649, -0.1715,  0.3257, -0.0145],
                        [-0.3347, -1.2013,  0.8388,  0.8044],
                        [-0.7819, -1.8453,  0.7127,  1.2772]])

In [4]:
t.a

tensor([[ 0.4342, -0.1405, -2.2848],
        [-0.5769,  0.6934, -0.6881]])

In [5]:
%timeit t.a

64 ns ± 0.019 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4bacbf9fa0>
├── 'a' --> tensor([[ 0.2424,  0.0303,  0.2222],
│                   [ 0.4765,  0.7179, -1.3499]])
└── 'x' --> <FastTreeValue 0x7f4bacc0d160>
    └── 'c' --> tensor([[ 0.4649, -0.1715,  0.3257, -0.0145],
                        [-0.3347, -1.2013,  0.8388,  0.8044],
                        [-0.7819, -1.8453,  0.7127,  1.2772]])

In [7]:
%timeit t.a = new_value

68.3 ns ± 0.0912 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4342, -0.1405, -2.2848],
               [-0.5769,  0.6934, -0.6881]]),
    x: Batch(
           c: tensor([[ 0.4649, -0.1715,  0.3257, -0.0145],
                      [-0.3347, -1.2013,  0.8388,  0.8044],
                      [-0.7819, -1.8453,  0.7127,  1.2772]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4342, -0.1405, -2.2848],
        [-0.5769,  0.6934, -0.6881]])

In [11]:
%timeit b.a

58.9 ns ± 0.0464 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7305, -0.2100, -1.6179],
               [-0.1487,  1.1739,  0.3374]]),
    x: Batch(
           c: tensor([[ 0.4649, -0.1715,  0.3257, -0.0145],
                      [-0.3347, -1.2013,  0.8388,  0.8044],
                      [-0.7819, -1.8453,  0.7127,  1.2772]]),
       ),
)

In [13]:
%timeit b.a = new_value

443 ns ± 0.227 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

816 ns ± 2.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

9.67 µs ± 22.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

140 µs ± 6.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

136 µs ± 7.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4afdacf880>
├── 'a' --> tensor([[[ 0.4342, -0.1405, -2.2848],
│                    [-0.5769,  0.6934, -0.6881]],
│           
│                   [[ 0.4342, -0.1405, -2.2848],
│                    [-0.5769,  0.6934, -0.6881]],
│           
│                   [[ 0.4342, -0.1405, -2.2848],
│                    [-0.5769,  0.6934, -0.6881]],
│           
│                   [[ 0.4342, -0.1405, -2.2848],
│                    [-0.5769,  0.6934, -0.6881]],
│           
│                   [[ 0.4342, -0.1405, -2.2848],
│                    [-0.5769,  0.6934, -0.6881]],
│           
│                   [[ 0.4342, -0.1405, -2.2848],
│                    [-0.5769,  0.6934, -0.6881]],
│           
│                   [[ 0.4342, -0.1405, -2.2848],
│                    [-0.5769,  0.6934, -0.6881]],
│           
│                   [[ 0.4342, -0.1405, -2.2848],
│                    [-0.5769,  0.6934, -0.6881]]])
└── 'x' --> <FastTreeValue 0x7f4afdacf280>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

28.8 µs ± 48.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4b08900eb0>
├── 'a' --> tensor([[ 0.4342, -0.1405, -2.2848],
│                   [-0.5769,  0.6934, -0.6881],
│                   [ 0.4342, -0.1405, -2.2848],
│                   [-0.5769,  0.6934, -0.6881],
│                   [ 0.4342, -0.1405, -2.2848],
│                   [-0.5769,  0.6934, -0.6881],
│                   [ 0.4342, -0.1405, -2.2848],
│                   [-0.5769,  0.6934, -0.6881],
│                   [ 0.4342, -0.1405, -2.2848],
│                   [-0.5769,  0.6934, -0.6881],
│                   [ 0.4342, -0.1405, -2.2848],
│                   [-0.5769,  0.6934, -0.6881],
│                   [ 0.4342, -0.1405, -2.2848],
│                   [-0.5769,  0.6934, -0.6881],
│                   [ 0.4342, -0.1405, -2.2848],
│                   [-0.5769,  0.6934, -0.6881]])
└── 'x' --> <FastTreeValue 0x7f4b0856afd0>
    └── 'c' --> tensor([[ 0.4649, -0.1715,  0.3257, -0.0145],
                        [-0.3347, -1.2013,  0.8388,  0.8044],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 50.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.8 µs ± 66.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.4342, -0.1405, -2.2848],
                [-0.5769,  0.6934, -0.6881]],
       
               [[ 0.4342, -0.1405, -2.2848],
                [-0.5769,  0.6934, -0.6881]],
       
               [[ 0.4342, -0.1405, -2.2848],
                [-0.5769,  0.6934, -0.6881]],
       
               [[ 0.4342, -0.1405, -2.2848],
                [-0.5769,  0.6934, -0.6881]],
       
               [[ 0.4342, -0.1405, -2.2848],
                [-0.5769,  0.6934, -0.6881]],
       
               [[ 0.4342, -0.1405, -2.2848],
                [-0.5769,  0.6934, -0.6881]],
       
               [[ 0.4342, -0.1405, -2.2848],
                [-0.5769,  0.6934, -0.6881]],
       
               [[ 0.4342, -0.1405, -2.2848],
                [-0.5769,  0.6934, -0.6881]]]),
    x: Batch(
           c: tensor([[[ 0.4649, -0.1715,  0.3257, -0.0145],
                       [-0.3347, -1.2013,  0.8388,  0.8044],
                       [-0.7819, -1.8453,  0.7127,  1.2772]],
         

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 582 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.4342, -0.1405, -2.2848],
               [-0.5769,  0.6934, -0.6881],
               [ 0.4342, -0.1405, -2.2848],
               [-0.5769,  0.6934, -0.6881],
               [ 0.4342, -0.1405, -2.2848],
               [-0.5769,  0.6934, -0.6881],
               [ 0.4342, -0.1405, -2.2848],
               [-0.5769,  0.6934, -0.6881],
               [ 0.4342, -0.1405, -2.2848],
               [-0.5769,  0.6934, -0.6881],
               [ 0.4342, -0.1405, -2.2848],
               [-0.5769,  0.6934, -0.6881],
               [ 0.4342, -0.1405, -2.2848],
               [-0.5769,  0.6934, -0.6881],
               [ 0.4342, -0.1405, -2.2848],
               [-0.5769,  0.6934, -0.6881]]),
    x: Batch(
           c: tensor([[ 0.4649, -0.1715,  0.3257, -0.0145],
                      [-0.3347, -1.2013,  0.8388,  0.8044],
                      [-0.7819, -1.8453,  0.7127,  1.2772],
                      [ 0.4649, -0.1715,  0.3257, -0.0145],
                      [-0.3347, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 270 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

344 µs ± 3.37 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
